In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.tools.plotting import autocorrelation_plot
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.arima_model import ARIMA, ARMAResults, ARIMAResults
from matplotlib.pyplot import acorr
import sys
print sys.version
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
%pylab inline

2.7.11 (default, Dec  5 2015, 14:44:47) 
[GCC 4.2.1 Compatible Apple LLVM 7.0.0 (clang-700.1.76)]
Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [17]:
# read in data from NYC Open Data
df = pd.read_json('https://data.cityofnewyork.us/resource/fhrw-4uyv.json')
print len(df)

1000


In [18]:
df.columns

Index([u'address_type', u'agency', u'agency_name', u'borough',
       u'bridge_highway_direction', u'bridge_highway_name',
       u'bridge_highway_segment', u'city', u'closed_date', u'community_board',
       u'complaint_type', u'created_date', u'cross_street_1',
       u'cross_street_2', u'descriptor', u'due_date', u'facility_type',
       u'incident_address', u'incident_zip', u'intersection_street_1',
       u'intersection_street_2', u'landmark', u'latitude', u'location',
       u'location_type', u'longitude', u'park_borough', u'park_facility_name',
       u'resolution_action_updated_date', u'resolution_description',
       u'road_ramp', u'school_address', u'school_city', u'school_code',
       u'school_name', u'school_not_found', u'school_number',
       u'school_phone_number', u'school_region', u'school_state',
       u'school_zip', u'status', u'street_name', u'taxi_company_borough',
       u'taxi_pick_up_location', u'unique_key', u'x_coordinate_state_plane',
       u'y_coordinate_

In [14]:
# we're only interested in the types of complaints and the dates they were reported
# so we will narrow down our dataframe to only include those features
data = df[['Created Date','descriptor']]
# to protect against any case sensitivity issues, let's make everything lower case
data.descriptor = [i.lower() for i in data.descriptor]


KeyError: "['descriptor'] not in index"

In [48]:
data.head()

,created_date,descriptor
0,2010-05-29T00:00:00.000,lighting
1,2010-05-29T00:00:00.000,walls
2,2010-05-29T00:00:00.000,smoke-detector
3,2010-05-29T00:00:00.000,bathtub
4,2010-05-29T00:00:00.000,vermin


In [31]:
print 'we have %d complaint descriptions, so we should probably pare them down' % len(data.descriptor.unique())

we have 123 complaint types, so we should probably pare them down


In [33]:
data[data['descriptor'].str.contains('loud')]

,created_date,descriptor
52,2010-05-30T04:05:01.000,loud music/party
69,2010-05-30T04:07:51.000,loud music/party
106,2010-05-30T12:37:13.000,loud music/party
109,2010-05-30T03:42:46.000,loud music/party
110,2010-05-30T03:46:39.000,loud music/party
111,2010-05-30T04:30:10.000,loud music/party
115,2010-05-30T05:56:29.000,loud talking
117,2010-05-30T02:38:00.000,loud music/party
118,2010-05-30T02:53:23.000,loud music/party
119,2010-05-30T02:57:24.000,loud music/party
